In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
batch_size = 64
block_size = 128
max_iter = 1000
# eval_interval = 2500
learning_rate = 1e-4
eval_iters = 100
embed_size = 384
n_layer = 4
n_head = 4
dropout = 0.2

cuda


In [3]:
DIR = os.getcwd()
filename = 'wizardofoz.txt'
database_path = os.path.join(DIR,'data',filename)
with open(database_path,'r',encoding='utf-8') as file:
    text = file.read()
chars = sorted(set(text))
vocab_size = len(chars)
print(chars,'\n',vocab_size)

['\n', ' ', '!', '&', '(', ')', ',', '-', '.', '0', '1', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '\ufeff'] 
 73


In [4]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda x: [ string_to_int[c] for c in x ]
decode = lambda x: ''.join( int_to_string[c] for c in x )

In [5]:
data = torch.tensor(encode(text), dtype=torch.int64)
print(data.shape)

torch.Size([207799])


In [6]:
train_size = int(0.8 * data.size(dim=0))
train_data = data[:train_size]
val_data = data[train_size:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    indxs = torch.randint(data.size(dim=0) - block_size, (batch_size,))
    x = torch.stack([ data[indx:indx+block_size] for indx in indxs ])
    y = torch.stack([ data[indx+1:indx+block_size+1] for indx in indxs ])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print("input: ")
print(x)
print("target: ")
print(y)

input: 
tensor([[ 1, 59, 48,  ..., 54,  8,  0],
        [58, 42, 52,  ..., 65,  1, 48],
        [ 1, 37, 49,  ..., 55, 54,  8],
        ...,
        [49, 44,  1,  ..., 61, 54, 44],
        [ 1, 55, 46,  ..., 48, 45, 53],
        [63, 41, 59,  ..., 44, 45, 44]], device='cuda:0')
target: 
tensor([[59, 48, 55,  ...,  8,  0,  0],
        [42, 52, 45,  ...,  1, 48, 41],
        [37, 49, 43,  ..., 54,  8,  0],
        ...,
        [44,  1, 60,  ..., 54, 44,  1],
        [55, 46,  1,  ..., 45, 53,  8],
        [41, 59,  1,  ..., 45, 44,  1]], device='cuda:0')


In [7]:
# class BigramLanguageModel(nn.Module):
#     def __init__(self, vocab_size):
#         super().__init__()
#         self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)
#     def forward(self, index, targets=None):
#         logits = self.token_embedding_table(index)
#         if targets is None:
#             loss = None
#         else:
#             B, T, C = logits.shape
#             logits = logits.view(B*T, C)
#             targets = targets.view(B*T)
#             loss = F.cross_entropy(logits, targets)
#         return logits, loss
#     def generate(self, index, max_new_tokens):
#         for _ in range(max_new_tokens):
#             logits, loss = self.forward(index)
#             logits = logits[:,-1,:]
#             probs = F.softmax(logits, dim=-1)
#             index_next = torch.multinomial(probs, num_samples=1)
#             index = torch.cat((index, index_next), dim=1)
#         return index


In [8]:
@torch.no_grad()
def estimate_loss():
    avg_loss = dict()
    model.eval()
    for split in ['train','val']:
        losses = torch.zeros((eval_iters,), dtype=torch.float16)
        for itr in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[itr] = loss.item()
        avg_loss[split] = losses.mean()
    model.train()
    return avg_loss

In [9]:
# model = BigramLanguageModel(vocab_size)
# m = model.to(device)

# context = torch.zeros((1,1), dtype=torch.int64, device=device)
# generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
# print(generated_chars)

In [10]:
# optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# for i in range(max_iter):

#     xb, yb = get_batch('train')
#     logits, loss = model.forward(xb, yb)
#     optimizer.zero_grad(set_to_none=True)
#     loss.backward()
#     optimizer.step()
#     if (i+1) % eval_iters == 0:
#         losses = estimate_loss()
#         print(f"Step [{(i+1)//eval_iters}/{max_iter//eval_iters}], train_loss: {losses['train']:.3f}, val_loss: {losses['val']:.3f}")

In [11]:
# context = torch.zeros((1,1), dtype=torch.int64, device=device)
# generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
# print(generated_chars)

In [12]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(embed_size, head_size)
        self.query = nn.Linear(embed_size, head_size)
        self.value = nn.Linear(embed_size, head_size)
        self.register_buffer('tril', torch.tril(torch.ones(block_size,block_size)))
        self.drop = nn.Dropout(dropout)
    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)
        out = k @ torch.transpose(q,-2,-1) * k.shape[-1]**0.5
        out = out.masked_fill(self.tril[:T,:T] == 0, float('-inf'))
        out = F.softmax(out, dim=-1)
        out = self.drop(out)
        out = out @ v
        return out
        

In [13]:
class Feedforward(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(embed_size, 4 * embed_size),
            nn.ReLU(),
            nn.Linear(4 * embed_size, embed_size),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)

In [14]:
class MultiHeadAttention(nn.Module):
    def __init__(self, n_head, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(n_head)])
        self.proj = nn.Linear(n_head * head_size, embed_size)
        self.drop = nn.Dropout(dropout)
    def forward(self, x):
        x = torch.cat([h(x) for h in self.heads], dim=-1)
        x = self.proj(x)
        x = self.drop(x)
        return x

In [15]:
class Block(nn.Module):
    def __init__(self, n_head):
        super().__init__()
        head_size = embed_size // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffnw = Feedforward()
        self.ln_1 = nn.LayerNorm(embed_size)
        self.ln_2 = nn.LayerNorm(embed_size)
    def forward(self, x):
        x_ = self.sa(x)
        x = self.ln_1(x_+x)
        x_ = self.ffnw(x)
        x = self.ln_2(x_+x)
        return x

In [16]:
class GPTLanguagemodel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, embed_size)
        self.position_embedding_table = nn.Embedding(block_size, embed_size)
        self.blocks = nn.Sequential(*[Block(n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(embed_size)
        self.lm_head = nn.Linear(embed_size, vocab_size)
        self.apply(self._init_weights)
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is None:
                nn.init.zeros_(module.bias)
        if isinstance(module, nn.Embedding):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)  
    def forward(self, index, targets=None):
        tok_emb = self.token_embedding_table(index)
        B, T, C = tok_emb.shape
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    def generate(self, index, max_new_token):
        for i in range(max_new_token):
            index_cond = index[:, -block_size:]
            logits, loss = self.forward(index_cond)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim=-1)
            new_index = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, new_index), dim=1)
        return index


model = GPTLanguagemodel(vocab_size)
m = model.to(device)

In [17]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for i in range(max_iter):

    xb, yb = get_batch('train')
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    if (i+1) % eval_iters == 0:
        losses = estimate_loss()
        print(f"Step [{(i+1)//eval_iters}/{max_iter//eval_iters}], train_loss: {losses['train']:.3f}, val_loss: {losses['val']:.3f}")

Step [1/10], train_loss: 2.432, val_loss: 2.436
Step [2/10], train_loss: 2.361, val_loss: 2.365
Step [3/10], train_loss: 2.324, val_loss: 2.334
Step [4/10], train_loss: 2.295, val_loss: 2.303
Step [5/10], train_loss: 2.246, val_loss: 2.254
Step [6/10], train_loss: 2.156, val_loss: 2.168
Step [7/10], train_loss: 2.049, val_loss: 2.076
Step [8/10], train_loss: 1.953, val_loss: 1.987
Step [9/10], train_loss: 1.881, val_loss: 1.929
Step [10/10], train_loss: 1.808, val_loss: 1.858


In [18]:
with open('model-01.pkl', 'wb') as f:
    pickle.dump(model, f)
print('model saved')

model saved


In [20]:
prompt = 'Hello! Can you see me?'
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context.unsqueeze(0), max_new_token=100)[0].tolist())
print(generated_chars)

Hello! Can you see me?” aiod Dorothy:

Dovy lome will scaupe halkiv it icth Foricerorog the Lion wender ut
onthirldin to t
